# Лабораторная работа 3

Создать сеть на базе LSTM используя TensorFlow (Keras). \
Сеть должна принимать на вход текстовый файл и на его базе генерировать свою абракадабру. \
Отчет должен содержать кроме кода, обучающий файл и результат генерации.

Сделана в Google Colab

In [141]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import string

Пример текста. Алиса в стране чудес

In [142]:
!wget https://fantasy-worlds.org/lib/id15523/download/txt -O alice_in_wonderland.txt

--2025-12-14 14:27:23--  https://fantasy-worlds.org/lib/id15523/download/txt
Resolving fantasy-worlds.org (fantasy-worlds.org)... 179.43.166.35, 2a02:7aa0:1619::38e6:5284
Connecting to fantasy-worlds.org (fantasy-worlds.org)|179.43.166.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306350 (299K) [application/zip]
Saving to: ‘alice_in_wonderland.txt’

alice_in_wonderland 100%[===================>] 299.17K   643KB/s    in 0.5s    

2025-12-14 14:27:24 (643 KB/s) - ‘alice_in_wonderland.txt’ saved [306350/306350]



Параметры

In [143]:
FILE_PATH = 'alice_in_wonderland.txt'
SEQUENCE_LENGTH = 50     # Длина входной последовательности

Загрузка и токенизация данных

In [144]:
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read().replace('\n', ' ')
    return text

def tokenize(text):
  tokens = text.split()
  table = str.maketrans('','',string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens

In [145]:
text = load_data(FILE_PATH)
tokens = tokenize(text)
print(f"Токены: {tokens[:10]}")
print(f"Количество токенов: {len(tokens)}")
print(f"Количество уникальных токенов: {len(set(tokens))}")

Токены: ['annotation', 'в', 'этой', 'книге', 'вы', 'встретитесь', 'с', 'девочкой', 'алисой', 'и']
Количество токенов: 25679
Количество уникальных токенов: 6582


Создание входных/выходных последовательностей

In [146]:
lines = []
for i in range(SEQUENCE_LENGTH, len(tokens)):
  seq = tokens[i-SEQUENCE_LENGTH: i]
  line = ' '.join(seq)
  lines.append(line)

print(f"\nКоличество входных последовательностей: {len(lines)}")


Количество входных последовательностей: 25629


Кодирование последовательностей

In [147]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)

In [148]:
tokenizer.word_index

{'и': 1,
 'не': 2,
 'в': 3,
 'что': 4,
 'алиса': 5,
 'она': 6,
 'на': 7,
 'я': 8,
 'а': 9,
 'как': 10,
 'это': 11,
 'с': 12,
 'все': 13,
 'так': 14,
 'он': 15,
 'но': 16,
 'сказала': 17,
 'ты': 18,
 'сказал': 19,
 'у': 20,
 'бы': 21,
 'было': 22,
 'тут': 23,
 'же': 24,
 'только': 25,
 'ее': 26,
 'вы': 27,
 'то': 28,
 'его': 29,
 'к': 30,
 'вот': 31,
 'очень': 32,
 'они': 33,
 'за': 34,
 'еще': 35,
 'ей': 36,
 'ни': 37,
 'да': 38,
 'ну': 39,
 'когда': 40,
 'по': 41,
 'из': 42,
 'мне': 43,
 'от': 44,
 'если': 45,
 'нет': 46,
 'была': 47,
 'там': 48,
 'уже': 49,
 'уж': 50,
 'король': 51,
 'подумала': 52,
 'был': 53,
 'кто': 54,
 'меня': 55,
 'шляпа': 56,
 'даже': 57,
 'или': 58,
 'деликатес': 59,
 'может': 60,
 'раз': 61,
 'королева': 62,
 'ведь': 63,
 'о': 64,
 'тоже': 65,
 'ничего': 66,
 'опять': 67,
 'чтобы': 68,
 'быть': 69,
 'грифон': 70,
 'тогда': 71,
 'про': 72,
 'начала': 73,
 'нее': 74,
 'хотя': 75,
 'конечно': 76,
 'ли': 77,
 'вдруг': 78,
 'до': 79,
 'мышь': 80,
 'алисе': 81,
 '

In [149]:
# Tokenizer резервирует индекс 0, поэтому фактический размер словаря = max_index + 1
vocab_size = len(tokenizer.word_index) + 1
print(f"Размер словаря: {vocab_size}")

Размер словаря: 6583


In [150]:
sequences = tokenizer.texts_to_sequences(lines)
sequences = np.array(sequences)
sequences

array([[6582,    3,  180, ..., 2365,  130, 1367],
       [   3,  180,  929, ...,  130, 1367,  130],
       [ 180,  929,   27, ..., 1367,  130, 1368],
       ...,
       [ 280,  103,    2, ...,  474,  930,    1],
       [ 103,    2,  923, ...,  930,    1,  326],
       [   2,  923,    9, ...,    1,  326,    1]])

Разделение

In [151]:
X, y = sequences[:, :-1], sequences[:, -1]
print(X)
print(y)

[[6582    3  180 ...  130 2365  130]
 [   3  180  929 ... 2365  130 1367]
 [ 180  929   27 ...  130 1367  130]
 ...
 [ 280  103    2 ...  473  474  930]
 [ 103    2  923 ...  474  930    1]
 [   2  923    9 ...  930    1  326]]
[1367  130 1368 ...    1  326    1]


y в one-hot векторы

In [152]:
y = to_categorical(y, num_classes=vocab_size)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [153]:
X.shape[1]

49

Построение модели

In [154]:
model = Sequential()

# Embedding Layer: преобразует индексы слов в плотные векторы
model.add(Embedding(vocab_size, 100))

# Слой LSTM: ядро модели для работы с последовательностями
model.add(LSTM(150))

# Слой Dropout для предотвращения переобучения
model.add(Dropout(0.2))

# Dense Layer: выходной слой с активацией softmax для получения вероятности следующего слова
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Обучение

In [155]:
model.fit(
    X, y,
    batch_size=256,
    epochs=50,
    verbose=1
)

Epoch 1/50
101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.0340 - loss: 8.1214
Epoch 2/50
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0420 - loss: 7.1677
Epoch 3/50
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0409 - loss: 7.0955
Epoch 4/50
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.0398 - loss: 7.0168
Epoch 5/50
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0397 - loss: 6.9662
Epoch 6/50
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.0410 - loss: 6.9252
Epoch 7/50
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.0455 - loss: 6.8806
Epoch 8/50
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0525 - loss: 6.8072
Epoch 9/50
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0563 - loss: 6.7304
Epoch 10/50
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.0590 - loss: 6.6445
Epoch 11/50
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0657 - loss: 6.5163
Epoch 12/50
101/101 ━━━━━━━━━━━━━━━━━━━━ 

Генерация текста

In [156]:
def generate_text(model, tokenizer, seed_text, n_words):
  result = []
  seed_text = seed_text.lower()
  for _ in range(n_words):
    # Кодирование входного текста
    encoded = tokenizer.texts_to_sequences([seed_text])[0]

    # Выравнивание (padding) последовательности до нужной длины
    # maxlen = SEQUENCE_LENGTH - 1, так как вход X имел длину 49
    padded = pad_sequences([encoded], maxlen = SEQUENCE_LENGTH - 1, truncating='pre')

    # Предсказание вероятностей следующего слова
    y_pred = model.predict(padded, verbose=0)

    # Получение индекса слова с наибольшей вероятностью
    predicted_word_index = np.argmax(y_pred[0])

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == predicted_word_index:
        predicted_word = word
        break

    seed_text += ' ' + predicted_word
    result.append(predicted_word)

  return ' '.join(result)


In [157]:
lines[len(lines)//2]

'все стороны как у морской подумала алиса бедный крошка пыхтел как паровоз вырываясь от своей новой няньки он то складывался вдвое то опять весь растопыривался алисе долго не удавалось взять его поудобнее как только ей это удалось наконец для чего пришлось завязать ребеночка узлом и крепко держать за правое ухо'

In [158]:
input = """алисе долго не удавалось взять его поудобнее и как только ей это удалось"""

result = generate_text(model, tokenizer, input, 20)
print(input, "->", result)

алисе долго не удавалось взять его поудобнее и как только ей это удалось -> выбраться как вдруг громкий свистящий звук заставил в книге сказал король и оба чудака до нее как ты раз когда
